# 大規模開発における機械学習について考える

# 直近の土日で考えていたこと
- 大規模開発における機械学習をやるには?
- 機械学習の広告配信に応用できるところ
- Scalaのメリットを活かした機械学習とは?

# 話す事
- 大規模開発をする上で、機械学習において、Scalaがいかに有用か

# 話さない事
- 機械学習の小難しいアルゴリズムについて

# 研究室を飛び出して使うためには?
大規模開発においては機械学習の精度よりも、
ビジネスロジックを根幹においた設計と、要求水準をみたす処理速度とともに、バグやエラーを出さない持続的な開発が可能な実装が求められる

## 機械学習と言えばpython

### よいところ
- ライブラリが豊富
- みんなつかってる
- 直感的な数値計算が可能
- GPUとの相性がよい
=> 研究など少人数でのプロジェクトに向いてる

### 大変なところ
- 動的型付け
- 非同期処理がやりづらい
- オブジェクト指向の良さを活かしたコードが書き辛い
=> 大規模開発には向いてない

In [ ]:
大規模開発においては機械学習の精度よりも、
ビジネスロジックを根幹においた設計と、要求水準をみたす処理速度とともに、バグやエラーを出さない持続的な開発が可能な実装が求められる

In [ ]:
# 機械学習のフェイズ

# なぜScalaなのか?
- 型安全 ()
- Option型、
- 非同期処理 
- 並列処理
- 関数型言語

# 数値計算はPythonに分がある?

# Breeze

![brezze](./Breeze.png)
さわやか  
Python(numpy)のように直感的な数値計算が可能になる  
Scala上でPythonライクな記法が可能。裏ではJavaを介してFortranを呼び出してる

In [19]:
classpath.add("org.scalanlp" %% "breeze" % "0.11.2")
classpath.add("org.scalanlp" %% "breeze-natives" % "0.11.2")

classpath.add("org.scalanlp" %% "breeze-viz" % "0.11.2")

Adding 0 artifact(s)
Adding 0 artifact(s)
Adding 9 artifact(s)


In [9]:
val v=DenseVector[Double](1,2,3,4,5)

v:* 2.0 //要素を二乗

//  一番目から三番目までにアクセス
v(0 to 3)

// 一番目から二番目間でアクセス python でいう[0:3]
v(1 until 3)

// 要素をリバース
v(v.length-1 to  0 by -1 )


// 0から10までの要素を持つベクトルの偶数の値だけ0にする
val vEven=linspace(0,1.0,11)
vEven(0 to 10 by 2):=0.0
vEven

v: DenseVector[Double] = DenseVector(1.0, 2.0, 3.0, 4.0, 5.0)
res8_1: DenseVector[Double] = DenseVector(2.0, 4.0, 6.0, 8.0, 10.0)
res8_2: DenseVector[Double] = DenseVector(1.0, 2.0, 3.0, 4.0)
res8_3: DenseVector[Double] = DenseVector(2.0, 3.0)
res8_4: DenseVector[Double] = DenseVector(5.0, 4.0, 3.0, 2.0, 1.0)
vEven: DenseVector[Double] = DenseVector(0.0, 0.1, 0.0, 0.30000000000000004, 0.0, 0.5, 0.0, 0.7000000000000001, 0.0, 0.9, 0.0)
res8_6: DenseVector[Double] = DenseVector(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
res8_7: DenseVector[Double] = DenseVector(0.0, 0.1, 0.0, 0.30000000000000004, 0.0, 0.5, 0.0, 0.7000000000000001, 0.0, 0.9, 0.0)

In [17]:
//関数型の特性も活かせる: Partital Functionと組み合わせて使う
DenseMatrix.tabulate(3, 2){case (i, j) => i+j}

res16: DenseMatrix[Int] = 0  1  
1  2  
2  3  

In [ ]:
# 機械学習のフェイズ再び

In [ ]:
# サンプル: 体重と身長で男女を区分する

In [22]:


import scala.reflect.ClassTag
import io.Source

import breeze.linalg._
import breeze.stats._
import breeze.optimize._

object HWData {
  
  val DataDirectory = "./"
  val fileName = "rep_height_weights.csv"
    
  def load:HWData =
  {
    val file = Source.fromFile(DataDirectory + fileName)
    val lines = file.getLines.toVector
    val splitLines = lines.map { _.split(',') }

    def fromList[T:ClassTag](index:Int, converter:(String => T)):DenseVector[T] =
      DenseVector.tabulate(lines.size) { irow => converter(splitLines(irow)(index)) }

    val genders = fromList(1, elem => elem.replace("\"", "").head)
    val weights = fromList(2, elem => elem.toDouble)
    val heights = fromList(3, elem => elem.toDouble)
    val reportedWeights = fromList(4, elem => elem.toDouble)
    val reportedHeights = fromList(5, elem => elem.toDouble)

    new HWData(weights, heights, reportedWeights, reportedHeights, genders)
  }

}

class HWData(
//     このようにそれぞれのデータに対応するベクトルをフィールドに持つように設計することで、Rなどと同じ記法でそれぞれのデータにアクセスできる
  val weights:DenseVector[Double],
  val heights:DenseVector[Double],
  val reportedWeights:DenseVector[Double],
  val reportedHeights:DenseVector[Double],
  val genders:DenseVector[Char]
) {

  val npoints = heights.length
  require(weights.length == npoints)
  require(reportedWeights.length == npoints)
  require(genders.length == npoints)
  require(reportedHeights.length == npoints)

  lazy val rescaledHeights:DenseVector[Double] =
    (heights - mean(heights)) / stddev(heights)

  lazy val rescaledWeights:DenseVector[Double] =
    (weights - mean(weights)) / stddev(weights)

  lazy val featureMatrix:DenseMatrix[Double] =
    DenseMatrix.horzcat( 
      DenseMatrix.ones[Double](npoints, 1), 
      rescaledHeights.toDenseMatrix.t,
      rescaledWeights.toDenseMatrix.t
    )

  lazy val target:DenseVector[Double] =
    genders.values.map { gender => if(gender == 'M') 1.0 else 0.0 }

  override def toString:String = s"HWData [ $npoints rows ]"

}

import scala.reflect.ClassTag
import io.Source
import breeze.linalg._
import breeze.stats._
import breeze.optimize._
defined object HWData
defined class HWData

可視化する(brreze-viz)

![データプロット](./data_plot.png)

身長と体重で男女が区別できそう

In [3]:
# 機械学習のフェイズ再び

import breeze.linalg._
v: breeze.linalg.DenseVector[Double] = DenseVector(1.0, 2.0, 3.0, 4.0, 5.0)
res2_2: breeze.linalg.DenseVector[Double] = DenseVector(1.0, 2.0, 3.0, 4.0)
res2_3: breeze.linalg.DenseVector[Double] = DenseVector(2.0, 3.0)
res2_4: breeze.linalg.DenseVector[Double] = DenseVector(5.0, 4.0, 3.0, 2.0, 1.0)

In [ ]:
数値計算はPythonに分がある?

# Breeze

![brezze](./Breeze.png)
さわやか
Python(numpy)のように直感的な数値計算が可能になる  
Scala上でPythonライクな記法が可能。裏ではJavaを介してFortranを呼び出してる

In [2]:
classpath.add("org.scalanlp" %% "breeze" % "0.11.2")
classpath.add("org.scalanlp" %% "breeze-natives" % "0.11.2")

Adding 0 artifact(s)
Adding 17 artifact(s)


In [6]:
val v=DenseVector[Double](1,2,3,4,5)

//  一番目から三番目までにアクセス
v(0 to 3)

// 一番目から二番目間でアクセス python でいう[0:3]
v(1 until 3)

// 要素をリバース
v(v.length-1 to  0 by -1 )


// 0から10までの要素を持つベクトルの偶数の値だけ0にする
val vEven=linspace(0,1.0,11)
vEven(0 to 10 by 2):=0.0
vEven

v: DenseVector[Double] = DenseVector(1.0, 2.0, 3.0, 4.0, 5.0)
res5_1: DenseVector[Double] = DenseVector(1.0, 2.0, 3.0, 4.0)
res5_2: DenseVector[Double] = DenseVector(2.0, 3.0)
res5_3: DenseVector[Double] = DenseVector(5.0, 4.0, 3.0, 2.0, 1.0)
vEven: DenseVector[Double] = DenseVector(0.0, 0.1, 0.0, 0.30000000000000004, 0.0, 0.5, 0.0, 0.7000000000000001, 0.0, 0.9, 0.0)
res5_5: DenseVector[Double] = DenseVector(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
res5_6: DenseVector[Double] = DenseVector(0.0, 0.1, 0.0, 0.30000000000000004, 0.0, 0.5, 0.0, 0.7000000000000001, 0.0, 0.9, 0.0)

In [ ]:
# 機械学習のフェイズ再び

In [ ]:
# サンプル: 体重と身長で男女を区分する

In [7]:


import scala.reflect.ClassTag
import io.Source

import breeze.linalg._
import breeze.stats._
import breeze.optimize._

object HWData {
  
  val DataDirectory = "./s4ds/chap02/data/"
  val fileName = "rep_height_weights.csv"
    
  def load:HWData =
  {
    val file = Source.fromFile(DataDirectory + fileName)
    val lines = file.getLines.toVector
    val splitLines = lines.map { _.split(',') }

    def fromList[T:ClassTag](index:Int, converter:(String => T)):DenseVector[T] =
      DenseVector.tabulate(lines.size) { irow => converter(splitLines(irow)(index)) }

    val genders = fromList(1, elem => elem.replace("\"", "").head)
    val weights = fromList(2, elem => elem.toDouble)
    val heights = fromList(3, elem => elem.toDouble)
    val reportedWeights = fromList(4, elem => elem.toDouble)
    val reportedHeights = fromList(5, elem => elem.toDouble)

    new HWData(weights, heights, reportedWeights, reportedHeights, genders)
  }

}

class HWData(
//     このようにそれぞれのデータに対応するベクトルをフィールドに持つように設計することで、Rなどと同じ記法でそれぞれのデータにアクセスできる
  val weights:DenseVector[Double],
  val heights:DenseVector[Double],
  val reportedWeights:DenseVector[Double],
  val reportedHeights:DenseVector[Double],
  val genders:DenseVector[Char]
) {

  val npoints = heights.length
  require(weights.length == npoints)
  require(reportedWeights.length == npoints)
  require(genders.length == npoints)
  require(reportedHeights.length == npoints)

  lazy val rescaledHeights:DenseVector[Double] =
    (heights - mean(heights)) / stddev(heights)

  lazy val rescaledWeights:DenseVector[Double] =
    (weights - mean(weights)) / stddev(weights)

  lazy val featureMatrix:DenseMatrix[Double] =
    DenseMatrix.horzcat( 
      DenseMatrix.ones[Double](npoints, 1), 
      rescaledHeights.toDenseMatrix.t,
      rescaledWeights.toDenseMatrix.t
    )

  lazy val target:DenseVector[Double] =
    genders.values.map { gender => if(gender == 'M') 1.0 else 0.0 }

  override def toString:String = s"HWData [ $npoints rows ]"

}

import scala.reflect.ClassTag
import io.Source
import breeze.linalg._
import breeze.stats._
import breeze.optimize._
defined object HWData
defined class HWData

In [24]:
import breeze.plot._
import breeze.linalg._

class ScatterplotMatrix(val figure:Figure) {
  
  def plotFeatures(featureMatrix:DenseMatrix[Double], labels:List[String]) {
    val ncols = featureMatrix.cols
    require(ncols == labels.size, 
      "Number of columns in feature matrix must match length of labels"
    )
    figure.clear
    figure.subplot(ncols, ncols, 0)
    (0 until ncols) foreach { irow => 
      (0 until irow) foreach { icol =>
        val p = selectPlot(ncols)(irow, icol)
        plotScatter(p)(
          featureMatrix(::, irow),
          featureMatrix(::, icol),
          labels(irow),
          labels(icol)
        )
      }
    }
  }

  private def plotScatter(plt:Plot)(
    xdata:DenseVector[Double], 
    ydata:DenseVector[Double], 
    xlabel:String, 
    ylabel:String) = {
      plt += plot(xdata, ydata, '.')
      plt.xlabel = xlabel
      plt.ylabel = ylabel
  }
    

  private def selectPlot(ncols:Int)(irow:Int, icol:Int) = {
    figure.subplot(ncols, ncols, (irow)*ncols + icol)
  }

}

import breeze.plot._
import breeze.linalg._
defined class ScatterplotMatrix

In [23]:
 

val data = HWData.load
  val fig = Figure("Scatterplot matrix demo")
  val m = new ScatterplotMatrix(fig)

  // Make a matrix with three columns: the height, weight and
  // reported weight data.
  val featureMatrix = DenseMatrix.horzcat(
    data.heights.toDenseMatrix.t,
    data.weights.toDenseMatrix.t,
    data.reportedWeights.toDenseMatrix.t
  )
  m.plotFeatures(featureMatrix, List("height", "weight", "reportedWeights"))

  fig.saveas("scatterplot_demo.png")

data: HWData = HWData [ 181 rows ]
fig: Figure = breeze.plot.Figure@7d29f239
m: ScatterplotMatrix = cmd19$$user$ScatterplotMatrix@2ecf5a79
featureMatrix: DenseMatrix[Double] = 182.0  77.0   77.0   
161.0  58.0   51.0   
161.0  53.0   54.0   
177.0  68.0   70.0   
157.0  59.0   59.0   
170.0  76.0   76.0   
167.0  76.0   77.0   
186.0  69.0   73.0   
178.0  71.0   71.0   
171.0  65.0   64.0   
175.0  70.0   75.0   
166.0  57.0   56.0   
161.0  51.0   52.0   
168.0  64.0   64.0   
163.0  52.0   57.0   
166.0  65.0   66.0   
187.0  92.0   101.0  
168.0  62.0   62.0   
197.0  76.0   75.0   
175.0  61.0   61.0   
...

In [ ]:
# 直近の土日で考えていたこと
- 大規模開発における機械学習をやるには?=>
- 機械学習の広告配信に応用できるところは?=>
- Scalaのメリットを活かした機械学習とは?=>